### Mecab 설치

In [1]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [2]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [3]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 460kB 44.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-02-15 07:13:35--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downlo

### import 및 setting

In [4]:
from keras.preprocessing import sequence
from keras.models import load_model

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score

from collections import defaultdict
import pickle, json, os
import re

from konlpy.tag import Mecab
mecab = Mecab()

# model load
model = load_model('/content/drive/MyDrive/STT_/model/Naver_dict_l3stm.h5')

# tokenizer load
with open("/content/drive/MyDrive/STT_/tokenizer/tokenizer3.pickle", 'rb') as fp:
    tokenizer = pickle.load(fp)

# token load
with open("/content/drive/MyDrive/STT_/token/token3.json", "r") as st_json:
    topic = json.load(st_json)

# stop words load
stop_words = [
        '가운데', '대체로', '간의', '크게', '같', '아니', '있', '되', '두', '받', '많', '크', '좋', '따르', '만들', '시키', '그러', '하나',
        '모르', '데', '자신', '어떤', '명', '앞', '번', '보이', '나', '어떻', '월', '들', '이렇', '점', '싶', '좀', '잘', '통하',
        '놓', '란', '이나', '것', '이', '그', '수', '등', '의', '때', '경우', '및', '를', '대한', '사용', '위', '때문', '약', '제', '후',
        '다른', '중', '이상', '일', '은', '위해', '가지', '시', '로', '세', '과', '다음', '두', '더', '또한', '함', '하나', '우리',
        '개', '관', '속', '가장', '모두', '점', '곳', '전', '또', '통해', '여러', '대해', '안', '즉', '모든', '내', '뒤', '보고', '이후',
        '데', '로서', '다시', '관련', '알', '비', '그것', '나', '일반', '각', '뜻', '정도', '사이', '사실', '도', '따라서', '고', '못', '예',
        '간', '동안', '매우', '정', '지금', '증', '임', '공', '해', '음', '앞', '번', '볼', '여기', '거나', '자', '부', '인', '날',
        '바로', '대부분', '기', '바', '주로', '뿐', '직접', '부터', '계속', '일부', '주', '재', '아래', '더욱', '초', '각각', '동시','권',
        '년', '곧', '온', '거의', '먼저', '비롯', '역시', '몇', '반드시', '거', '보', '단', '듯', '가장', '서로', '모든', '에 대한 ',
        ' 수 ', '모두', '의', '에', '대해', '그런데', '으로', '이것', '대로', '것', '그대로', '그리고', '것도', '수가', '이 ', '해야',
        '지금', '할 수가', '할 수', '각종', '요게', '여기', '혹시', '우리', '한번', '이번', '당신', '이렇게', '차고', '어떻게', '뭐', '깜짝', '거지',
        '싶어서', '그래서', '정말', '이런', '도저히', '거죠', '하면은', '이제', '그렇게', '그럼', '많이', '이거', '그거', '저거', '누가',
        '그래', '그냥', '바로', '누가', '다시', '그래도', '간단히', '거야', '이따', '00', '근데', '결국', '이때', '누가', '그런', '딱', '일단',
        '보면', '하나', '어디', '부터', '원', '위하', '나오', '중', '못하', '그렇', '오', '대하', '한', '지', '하']

max_word =20000
max_len = 1000 #max_len = 500

In [19]:
## 받아올 것 -> API 부분임
str_ = u'''
교활(狡猾)과 낭패(狼狽)는 상상의 동물 이름이다
이 교활이란 놈은 어찌나 간사한지 여우를 능가할 정도인데, 중국의 기서(奇書)인 『산해경(山海經)』에 등장하는 동물이다
교(狡)라는 놈은 모양은 개인데 온몸에 표범의 무늬가 있으며, 머리에는 쇠뿔을 달고 있다고 한다
이놈이 나타나면 그해에는 대풍(大豊)이 든다고 하는데, 이 녀석이 워낙 간사하여 나올 듯 말 듯 애만 태우다가 끝내 나타나지 않는다고 한다 
한편 이 교의 친구로 활(猾)이라는 놈이 있는데 이놈은 교보다 더 간악하다
이놈은 생김새는 사람 같은데 온몸에 돼지털이 숭숭 나 있으며 동굴 속에 살면서 겨울잠을 잔다
도끼로 나무를 찍는 듯한 소리를 내는데, 이놈이 나타나면 온 천하가 대란(大亂)에 빠진다고 한다
이처럼 교와 활은 간악하기로 유명한 동물인데, 길을 가다가 호랑이라도 만나면 몸을 똘똘 뭉쳐 조그만 공처럼 변신하여 제 발로 호랑이 입속으로 뛰어들어 내장을 마구 파먹는다
호랑이가 그 아픔을 참지 못해 뒹굴다가 죽으면 그제야 유유히 걸어나와 미소를 짓는다
여기에서 바로 그 ‘교활한 미소’라는 관용구가 생겨났다

메탄올(methanol 또는 methyl alcohol) 또는 메틸 알코올이라고 부르며 나무로부터 많이 얻어진다고 하여 목정(wood spirit) 이라고도 한다
1661년에 보일(Robert Boyle)에 의해 회양목의 증류 과정에서 최초로 분리되었다
MeOH로 쓰기도 하며, 알코올 중에서 가장 간단한 구조로 되어 있다
물보다 가볍고, 무색의 가연성이 있는 극성을 띠는 액체이며, 맛과 냄새는 술의 주성분인 에탄올과 비슷하다
부동액, 연료 등으로 쓰이며 다양한 화학 반응에서 용매 또는 주요한 출발 물질로 사용된다
'''

nums = 2


### PreProcessing

In [20]:
str_ = re.sub('[^A-Za-z0-9가-힣.]', ' ', str_)
str_ = re.sub(' +', ' ', str_)

In [21]:
sen_tag = []  # tagging job
morph = mecab.pos(str_)
print(morph)

[('교활', 'NNG'), ('과', 'NNG'), ('낭', 'NNG'), ('패', 'VV'), ('는', 'ETM'), ('상상', 'NNG'), ('의', 'JKG'), ('동물', 'NNG'), ('이름', 'NNG'), ('이', 'VCP'), ('다', 'EF'), ('이', 'MM'), ('교활', 'NNG'), ('이', 'VCP'), ('란', 'ETM'), ('놈', 'NNB'), ('은', 'JX'), ('어찌나', 'MAG'), ('간사', 'XR'), ('한지', 'XSA+EC'), ('여우', 'NNG'), ('를', 'JKO'), ('능가', 'NNG'), ('할', 'XSV+ETM'), ('정도', 'NNG'), ('인데', 'VCP+EC'), ('중국', 'NNP'), ('의', 'JKG'), ('기서', 'NNG'), ('인', 'VCP+ETM'), ('산해경', 'NNP'), ('에', 'IC'), ('등장', 'NNG'), ('하', 'XSV'), ('는', 'ETM'), ('동물', 'NNG'), ('이', 'JKS'), ('다', 'MAG'), ('교', 'NNG'), ('라는', 'VCP+ETM'), ('놈', 'NNB'), ('은', 'JX'), ('모양', 'NNG'), ('은', 'JX'), ('개', 'NNG'), ('인데', 'VCP+EC'), ('온몸', 'NNG'), ('에', 'JKB'), ('표범', 'NNG'), ('의', 'JKG'), ('무늬', 'NNG'), ('가', 'JKS'), ('있', 'VA'), ('으며', 'EC'), ('머리', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('쇠뿔', 'NNG'), ('을', 'JKO'), ('달', 'VV'), ('고', 'EC'), ('있', 'VX'), ('다고', 'EC'), ('한다', 'VX+EF'), ('이놈', 'NP'), ('이', 'JKS'), ('나타나', 'VV'), ('면', 'EC'), ('그해', 'N

In [22]:
new_str = ''
verb_1 = ['다', '요', '죠', '까', '니']
for word, tag in morph:
  if 'EF' in tag or 'VX' in tag or 'EC' in tag:
    if word[-1] in verb_1:
      new_str += word +'.'
    else:
      new_str += word + ' '
  else:
    new_str += word + ' '
new_str

# 형태소 분석기로 부족한 동사들 dotword로 충전
dotword = ['립니다', '까요', '니다', '었다', '해요', '혔다', '였다', '든요', '거든요', '아요', 
            '습니다', '합니다', '입니다', '랍니다', '씁시다', '합시다','하죠', '보죠', '이죠', '렇죠', 
            '시죠', '었고요', '였고요', '니고요', '렇고요', '니까요', '는데>요', '니까', '이에요', '잖아요', 
            '고요', '구요','게요']

for i in dotword:
  new_str = new_str.replace(i + ' ', i + '.')
new_str

'교활 과 낭 패 는 상상 의 동물 이름 이 다.이 교활 이 란 놈 은 어찌나 간사 한지 여우 를 능가 할 정도 인데 중국 의 기서 인 산해경 에 등장 하 는 동물 이 다 교 라는 놈 은 모양 은 개 인데 온몸 에 표범 의 무늬 가 있 으며 머리 에 는 쇠뿔 을 달 고 있 다고 한다.이놈 이 나타나 면 그해 에 는 대풍 이 든다고 하 는데 이 녀석 이 워낙 간사 하 여 나올 듯 말 듯 애 만 태우 다가 끝내 나타나 지 않 는다고 한다.한 편 이 교 의 친구 로 활 이 라는 놈 이 있 는데 이놈 은 교 보다 더 간악 하 다.이놈 은 생김새 는 사람 같 은데 온몸 에 돼지 털 이 숭숭 나 있 으며 동굴 속 에 살 면서 겨울잠 을 잔다.도끼 로 나무 를 찍 는 듯 한 소리 를 내 는데 이놈 이 나타나 면 온 천하 가 대란 에 빠진다고 한다.이 처럼 교와 활 은 간악 하 기 로 유명 한 동물 인데 길 을 가 다가 호랑이 라도 만나 면 몸 을 똘똘 뭉쳐 조그만 공 처럼 변신 하 여 제 발 로 호랑이 입속 으로 뛰어들 어 내장 을 마구 파먹 는다.호랑이 가 그 아픔 을 참 지 못해 뒹굴 다가 죽 으면 그제야 유유히 걸 어 나와 미소 를 짓 는다.여기 에서 바로 그 교활 한 미소 라는 관용구 가 생겨났 다.메탄올 methanol 또는 methyl alcohol 또는 메틸 알코올 이 라고 부르 며 나무 로부터 많이 얻 어 진다고 하 여 목정 wood spirit 이 라고 도 한다.1661 년 에 보일 Robert Boyle 에 의해 회양목 의 증류 과정 에서 최초 로 분리 되 었 다.MeOH 로 쓰 기 도 하 며 알코올 중 에서 가장 간단 한 구조 로 되 어 있 다.물 보다 가볍 고 무색 의 가연 성 이 있 는 극성 을 띠 는 액체 이 며 맛 과 냄새 는 술 의 주성분 인 에탄올 과 비슷 하 다.부동액 연료 등 으로 쓰이 며 다양 한 화학 반응 에서 용매 또는 주요 한 출발 물질 로 사용 된다.'

In [24]:
documents = new_str.split('.')
new_docs = []
for i in documents[:-1]:
  temp = ''
  for j in mecab.nouns(i):
    if j not in stop_words:
      temp += j + ' '
  new_docs.append(temp)
new_docs

['교활 낭 상상 동물 이름 ',
 '교활 놈 여우 능가 중국 기서 산해경 등장 동물 교 놈 모양 온몸 표범 무늬 머리 쇠뿔 ',
 '이놈 그해 대풍 녀석 간사 애 만 ',
 '편 교 친구 활 놈 이놈 교 간악 ',
 '이놈 생김 사람 온몸 돼지 동굴 겨울잠 ',
 '도끼 나무 소리 이놈 천하 가 대란 ',
 '교와 활 간악 동물 길 호랑이 몸 변신 발 호랑이 입속 내장 ',
 '호랑이 아픔 미소 ',
 '교활 미소 관용구 ',
 '메탄올 메틸 알코올 나무 목정 ',
 '회양목 증류 과정 최초 분리 ',
 '알코올 구조 ',
 '물 무색 가연 극성 액체 맛 냄새 술 주성분 에탄올 ',
 '부동액 연료 다양 화학 반응 용매 출발 물질 ']

### k-means clustering -> 문단 나누기

In [25]:
paragraphs = []
paragraphs2 = []
paragraphs3 = []
paragraphs4 = []

In [26]:
# tfidf -> 단어 vector화
vectorizer = TfidfVectorizer(max_features = max_word)
sp_matrix = vectorizer.fit_transform(new_docs)
sp_matrix

<14x66 sparse matrix of type '<class 'numpy.float64'>'
	with 79 stored elements in Compressed Sparse Row format>

In [27]:
Kmodel = KMeans(n_clusters=nums, init='k-means++', max_iter=800, n_init=4)
Kmodel.fit(sp_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=800,
       n_clusters=2, n_init=4, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [28]:
order_centroids = Kmodel.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [29]:
for i in range(nums):
  paragraphs.append([x for x, y in zip(documents, Kmodel.labels_) if  y == i])
  paragraphs2.append([x for x, y in zip(new_docs, Kmodel.labels_) if y == i])

In [30]:
paragraphs2

[['교활 낭 상상 동물 이름 ',
  '교활 놈 여우 능가 중국 기서 산해경 등장 동물 교 놈 모양 온몸 표범 무늬 머리 쇠뿔 ',
  '이놈 그해 대풍 녀석 간사 애 만 ',
  '편 교 친구 활 놈 이놈 교 간악 ',
  '이놈 생김 사람 온몸 돼지 동굴 겨울잠 ',
  '도끼 나무 소리 이놈 천하 가 대란 ',
  '메탄올 메틸 알코올 나무 목정 ',
  '회양목 증류 과정 최초 분리 ',
  '알코올 구조 ',
  '물 무색 가연 극성 액체 맛 냄새 술 주성분 에탄올 ',
  '부동액 연료 다양 화학 반응 용매 출발 물질 '],
 ['교와 활 간악 동물 길 호랑이 몸 변신 발 호랑이 입속 내장 ', '호랑이 아픔 미소 ', '교활 미소 관용구 ']]

In [31]:
for i in range(len(paragraphs2)):
  temp,temp2 = '', ''
  for j in paragraphs[i]:
    temp += j
  for j in paragraphs2[i]:
    temp2 += j
  paragraphs3.append(temp)
  paragraphs4.append(temp2.split()[:-1])

In [32]:
del paragraphs2

### keyword 추출

In [41]:
def rmmult(sort_):
  lili = []
  for i in range(len(sort_)):
      if (sort_[i] in lili) or (sort_[i][1] < 0.01):
          continue
      else:
          lili.append(sort_[i])
  return lili

def rmmult_noarr(sort_):
    lili = []
    for i in range(len(sort_)):
        if (sort_[i][0] in lili) or (sort_[i][1] == 0):
            continue
        else:
            lili.append(sort_[i][0])
    return lili

In [54]:
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(documents)
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
  word2id[feature] = idx

li = []

for i, sentence in enumerate(documents):
  morph = mecab.nouns(sentence)
  a = [(token, sp_matrix[i, word2id[token]]) for token in morph if token not in stop_words]
  sortt = sorted(a, key=lambda k:k[1])
  sortt = rmmult(sortt)

  li += [(token, sp_matrix[i, word2id[token]]) for token in morph]

  sort_li = sorted(li, key=lambda k: k[1])
a = sort_li[::-1]

In [60]:
keywords = [word for word, score in a[:10]]

In [61]:
keywords

['이름', '상상', '친구', '구조', '동물', '교활', '간악', '관용구', '여기', '알코올']

### model predcit

In [57]:
subject_classifier = []
topic_idx = []
for i in range(nums):
  sequences = tokenizer.texts_to_sequences([paragraphs4[i]])
  sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
  topic_idx.append(model.predict_classes(sequences_matrix))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [58]:
topic_idx

[array([64]), array([25])]

### model 결과 및 전달 할 것 3개

In [62]:
for i in range(nums):
  print("----------------paragraph-----------------")
  print(paragraphs[i])

  print("----------------subject_classifier-----------------")
  print(topic[str(int(topic_idx[i]))])

  print()
  print()

print("----------------keyword-----------------")
print(keywords)

  
  

----------------paragraph-----------------
['교활 과 낭 패 는 상상 의 동물 이름 이 다', '이 교활 이 란 놈 은 어찌나 간사 한지 여우 를 능가 할 정도 인데 중국 의 기서 인 산해경 에 등장 하 는 동물 이 다 교 라는 놈 은 모양 은 개 인데 온몸 에 표범 의 무늬 가 있 으며 머리 에 는 쇠뿔 을 달 고 있 다고 한다', '이놈 이 나타나 면 그해 에 는 대풍 이 든다고 하 는데 이 녀석 이 워낙 간사 하 여 나올 듯 말 듯 애 만 태우 다가 끝내 나타나 지 않 는다고 한다', '한 편 이 교 의 친구 로 활 이 라는 놈 이 있 는데 이놈 은 교 보다 더 간악 하 다', '이놈 은 생김새 는 사람 같 은데 온몸 에 돼지 털 이 숭숭 나 있 으며 동굴 속 에 살 면서 겨울잠 을 잔다', '도끼 로 나무 를 찍 는 듯 한 소리 를 내 는데 이놈 이 나타나 면 온 천하 가 대란 에 빠진다고 한다', '메탄올 methanol 또는 methyl alcohol 또는 메틸 알코올 이 라고 부르 며 나무 로부터 많이 얻 어 진다고 하 여 목정 wood spirit 이 라고 도 한다', '1661 년 에 보일 Robert Boyle 에 의해 회양목 의 증류 과정 에서 최초 로 분리 되 었 다', 'MeOH 로 쓰 기 도 하 며 알코올 중 에서 가장 간단 한 구조 로 되 어 있 다', '물 보다 가볍 고 무색 의 가연 성 이 있 는 극성 을 띠 는 액체 이 며 맛 과 냄새 는 술 의 주성분 인 에탄올 과 비슷 하 다', '부동액 연료 등 으로 쓰이 며 다양 한 화학 반응 에서 용매 또는 주요 한 출발 물질 로 사용 된다']
----------------subject_classifier-----------------
한국문학


----------------paragraph-----------------
['이 처럼 교와 활 은 간악 하 기 로 유명 한 동물 인데 길 을 가 다가 호랑이 라도 만나 면 몸 을 똘똘 뭉쳐 조그만 공 처